## 01. 라이브러리 로드

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pykrx import stock
import time

## 02. 데이터 로드
- k_df1 : 2011 ~ 2024 티커, 로그수익률, 연도
- k_df2 : 2012 ~ 2024, 날짜, kospi_log_return, kosdqa_log_return 

In [11]:
k_df1 = pd.read_csv('data/krx/KOSPI_KOSDAQ_log_returns_2011_2024.csv')
k_df2 =  pd.read_csv('data/krx/KOSPI_KOSDAQ_log_returns_2012_2024.csv')

#### k_df1을 기준으로 left 조인

In [12]:
df = pd.merge(k_df1, k_df2, on=['날짜'], how='left')

## 03. 시장구분 컬럼을 위해 티커 수집
- 연도 범위 설정
- 티커를 키로 시장 구분을 값으로 딕셔너리 생성
- 각 연도를 순회하며 티커와 시장 정보 수집
- 최종 결과 DataFrame으로 변환

In [14]:
years = range(2012, 2025) # 2012년부터 2024년까지

ticker_market_map = {}

print("2012년부터 2024년까지 모든 상장 종목과 시장 정보를 수집합니다.")
print("-" * 50)

for year in years:
    base_date = f"{year}0101"
    
    try:
        print(f"{year}년 기준 종목 수집 중...")
        
        # 코스피 티커 리스트 조회 및 딕셔너리에 추가
        kospi_tickers = stock.get_market_ticker_list(date=base_date, market="KOSPI")
        for ticker in kospi_tickers:
            ticker_market_map[ticker] = 'KOSPI'
        
        # 코스닥 티커 리스트 조회 및 딕셔너리에 추가
        kosdaq_tickers = stock.get_market_ticker_list(date=base_date, market="KOSDAQ")
        for ticker in kosdaq_tickers:
            ticker_market_map[ticker] = 'KOSDAQ'
            
        time.sleep(0.5) # 서버 부하 방지를 위한 지연

    except Exception as e:
        print(f"{year}년 데이터 수집 중 오류 발생: {e}")

print("-" * 50)
print("모든 정보 수집 완료!")


# 딕셔너리의 아이템들을 리스트로 만들어 데이터프레임 생성
df_all_tickers = pd.DataFrame(
    list(ticker_market_map.items()),
    columns=['티커', '시장구분']
)

# 티커 번호 순으로 정렬
df_all_tickers = df_all_tickers.sort_values(by='티커').reset_index(drop=True)

2012년부터 2024년까지 모든 상장 종목과 시장 정보를 수집합니다.
--------------------------------------------------
2012년 기준 종목 수집 중...
2013년 기준 종목 수집 중...
2014년 기준 종목 수집 중...
2015년 기준 종목 수집 중...
2016년 기준 종목 수집 중...
2017년 기준 종목 수집 중...
2018년 기준 종목 수집 중...
2019년 기준 종목 수집 중...
2020년 기준 종목 수집 중...
2021년 기준 종목 수집 중...
2022년 기준 종목 수집 중...
2023년 기준 종목 수집 중...
2024년 기준 종목 수집 중...
--------------------------------------------------
모든 정보 수집 완료!


#### 시장구분 컬럼 생성

In [15]:
df['시장구분'] = df['티커'].map(ticker_market_map)

#### 시장_로그수익률 컬럼 생성
- 로그수익률, 시장_로그수익률 결측치 제거

In [16]:
df['시장_로그수익률'] = np.where(
    df['시장구분'] == 'KOSPI',  # 만약 '시장구분'이 'KOSPI'이면
    df['KOSPI_log_return'],        # 코스피 수익률을 적용하고
    df['KOSDAQ_log_return']         # 그렇지 않으면 코스닥 수익률을 적용
)

df.dropna(subset=['로그수익률', '시장_로그수익률'], inplace=True)

#### 날짜 컬럼 재정의

In [17]:
df['날짜'] = pd.to_datetime(df['날짜'])
df.set_index('날짜', inplace=True)

## 04. 베타 계산
- 각 날짜가 어떤 기간에 속하는지 라벨링 (4월 말 기준)
- 각 기간에 해당하는 실제 기간 시작일 매핑

In [18]:
# 각 날짜가 어떤 '기간'에 속하는지 라벨링 (4월 말 기준)
period_year = np.where(df.index.month > 4, df.index.year, df.index.year - 1)
df['period_year'] = period_year

# 각 period_year에 해당하는 실제 '기간 시작일' 매핑
period_start_dates = {
    year: pd.to_datetime(f'{year}-04-01') + pd.offsets.BMonthEnd()
    for year in range(df['period_year'].min(), df['period_year'].max() + 1)
}
df['period_start_date'] = df['period_year'].map(period_start_dates)

#### 베타 계산을 위한 calculate_beta 함수 정의
- 공분산 계산
- 시장 분산 계산
- 베타 계산

In [19]:
def calculate_beta(group):
    # 공분산과 분산을 계산하기에 데이터가 충분한지 확인 (최소 2개)
    if len(group) < 2:
        return np.nan
    # 공분산 계산
    covariance = group['로그수익률'].cov(group['시장_로그수익률'])
    # 시장 분산 계산
    market_variance = group['시장_로그수익률'].var()
    
    # 시장 분산이 0인 경우 베타 계산 불가
    if market_variance == 0:
        return np.nan
    # 베타 계산    
    beta = covariance / market_variance
    return beta

#### 기간 시작일과 티커로 그룹화 후 베타 계산

In [20]:
beta_results = df.groupby(['period_start_date', '티커']).apply(calculate_beta)

C:\Users\tmddn\AppData\Local\Temp\ipykernel_15252\648155991.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  beta_results = df.groupby(['period_start_date', '티커']).apply(calculate_beta)


## 05. 결과 확인 및 데이터 저장
- 시작일 : 컬럼명 재정의
- 종료일 : 시작일에 1년을 더해 종료일 생성
- 컬럼 순서 정리

In [21]:
results_df = beta_results.reset_index(name='베타')
results_df.rename(columns={'period_start_date': '시작일'}, inplace=True)

# 시작일에 1년을 더해 종료일 생성 (4월 말 기준)
results_df['종료일'] = results_df['시작일'].apply(
    lambda date: pd.to_datetime(f'{date.year + 1}-04-01') + pd.offsets.BMonthEnd()
)
results_df = results_df[['시작일', '종료일', '티커', '베타']]
results_df

,시작일,종료일,티커,베타
0,2011-04-29,2012-04-30,000060,0.666369
1,2011-04-29,2012-04-30,000327,-0.047741
2,2011-04-29,2012-04-30,000360,0.619235
3,2011-04-29,2012-04-30,000365,0.286917
4,2011-04-29,2012-04-30,000420,0.728451
...,...,...,...,...
30846,2024-04-30,2025-04-30,950170,0.389944
30847,2024-04-30,2025-04-30,950190,0.565052
30848,2024-04-30,2025-04-30,950200,0.838629
30849,2024-04-30,2025-04-30,950210,1.088839


#### 컬럼 맞추기
- 회계년도 : 시작일의 년도 추출하여 생성
- 시작일과, 종료일 컬럼 제거

In [22]:
results_df['회계년도'] = results_df['시작일'].dt.year
results_df.drop(columns=['시작일', '종료일'], inplace=True)
results_df.sort_values(by=['티커', '회계년도'], inplace=True)

#### 데이터 저장

In [24]:
results_df.to_excel("data/krx/beta.xlsx", index=False)